In [11]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark

In [12]:
mutable struct mLCP
  c0      :: Float64          # constant term in objective
  q1      :: Vector           # c
  q2      :: Vector            # -b
  M11     :: Array
  M12     :: Array
  M21     :: Array     # A
  M22     :: Array
  lvar    :: Vector
  uvar    :: Vector
end


In [36]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1
        end
        if !(lvar[i] < x_tilde[i] < uvar[i])
            x_tilde[i] = (lvar[i] + uvar[i])/2
        end
    end

    return x_tilde
end
    



init_x0_lsq (generic function with 1 method)

In [37]:
function compute_α_dual(v, dir_v)
    n = length(v)
    if n == 0
        return 1.
    end
    α = 1.
    for i=1:n
        if dir_v[i] < 0
            α_new = -v[i] * 0.999 / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end


    
function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    α_l, α_u = 1., 1.
    for i=1:n
        if dir_v[i] > 0
            α_u_new = (uvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < 0
            α_l_new = (lvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end

function compute_μ(x_l, x_u, s_l, s_u, lvar, uvar, nb_low, nb_upp)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s_l' * (x_l-lvar) + s_u' * (uvar-x_u)) / (nb_low + nb_upp)
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [38]:
function mehrotraPCQuadBounds(mLCP, max_iter; ϵ=1e-8, tol_Δx=1e-8, ϵ_μ=1e-8, display=true)
    
    start_time = time()
    elapsed_time = 0.0
    
    # get variables from QuadraticModel
    lvar, uvar = mLCP.lvar, mLCP.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp =  findall((x -> x!=-Inf), lvar),  findall((x -> x!=Inf), uvar) # finite bounds index
    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints
    M12 = mLCP.M12
    M21 = mLCP.M21
    n_rows, n_cols = size(M21) 
    M11 = mLCP.M11
    M22 = mLCP.M22
    q1 = mLCP.q1
    q2 = mLCP.q2
    c0 = mLCP.c0
    
    x = init_x0_lsq(M21, -q2, lvar, uvar)
    
    @assert all(x .> lvar) && all(x .< uvar)
    s_l, s_u = copy(Oc), copy(Oc)
    s_l[ilow] .= 1.
    s_u[iupp] .= 1.
    
    M11x = M11 * x
    λ = M12 \ (q1+M11x) # least square initialisation, s_0 = stilde_0
    
    
    r1 = -M11x + M12 * λ + s_l - s_u - q1
    r2 = M21 * x + q2 - M22 * λ
    μ = compute_μ(x[ilow], x[iupp], 
                  s_l[ilow], s_u[iupp], 
                  lvar[ilow], uvar[iupp],
                  n_low, n_upp)

    k = 0
    n_1_p1 =  norm(q1) + 1.
    n_2_p1 =  norm(q2) + 1.
    
    # matrices without infinity constraints
    X_low = Diagonal(x[ilow])
    S_low = spzeros(n_low, n_cols)
    I_low = spzeros(n_cols, n_low)
    for j=1:n_low   # cartesian index not working when nb_non_inf_l = 0
        S_low[j, ilow[j]] = s_l[ilow[j]]
        I_low[ilow[j], j] = 1
    end
    
    X_upp = Diagonal(x[iupp])
    S_upp = spzeros(n_upp, n_cols)
    I_op_upp = spzeros(n_cols, n_upp)
    for j=1:n_upp 
        S_upp[j, iupp[j]] = s_u[iupp[j]]
        I_op_upp[iupp[j], j] = -1
    end
    Lvar_low = Diagonal(lvar[ilow]) 
    Uvar_upp = Diagonal(uvar[iupp])
    
    Orl = spzeros(n_rows, n_low)
    Oru = spzeros(n_rows, n_upp)
    Oul = spzeros(n_upp,n_low)
    Our = Oru'
    Olu = Oul'
    Olr = Orl'
    Orc1 = zeros(n_rows+n_cols, 1)

    # stopping criterion
    xTM11x = x' * M11x
    q1Tx = q1' * x
    pri_obj = xTM11x/2 + q1Tx + c0
    dual_obj = -q2' * λ - xTM11x/2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0
    pdd = abs(pri_obj - dual_obj ) / (1 + abs(pri_obj)) 
    cond_r2, cond_r1 = norm(r2) / n_2_p1, norm(r1) / n_1_p1
    optimal = pdd < ϵ && cond_r2 < ϵ && cond_r1 < ϵ # start false?
    small_Δx, small_μ = false, μ < ϵ_μ

    # display
    if display == true
        println("Iter    primal      pdd      rb cond    rc cond    step x       μ")
        @printf("%4d  % 9.2e   %7.1e    %7.1e    %7.1e    %7.1e   %7.1e\n", 
                k, pri_obj, pdd, cond_r2, cond_r1,0., μ)
    end
    

    while k<max_iter && !optimal && !small_Δx && !small_μ
        
        # Affine scaling direction
        J = [ -M11         M12         I_low             I_op_upp
               M21        -M22          Orl                Oru
             S_low         Olr     X_low-Lvar_low          Olu
             S_upp         Our          Oul            X_upp-Uvar_upp ]
        LU = lu(J)
        F_aff = [-r1
                 -r2
                 -(x[ilow]-lvar[ilow]).*s_l[ilow]
                 -(x[iupp]-uvar[iupp]).*s_u[iupp]]

        Δ_aff = LU\F_aff
        
        α_aff_pri = compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)
        
        μ_aff = compute_μ(x[ilow] + α_aff_pri * Δ_aff[1:n_cols][ilow],
                          x[iupp] + α_aff_pri * Δ_aff[1:n_cols][iupp],
                          s_l[ilow] + α_aff_dual_final * Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low],
                          s_u[iupp] + α_aff_dual_final * Δ_aff[n_rows+n_cols+n_low+1: end],
                          lvar[ilow], uvar[iupp],
                          n_low, n_upp)
        
        σ = (μ_aff / μ)^3

        # corrector and centering step
        F_cc = [Orc1
                σ*μ .- Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
                -σ*μ .- Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]] 

        Δ_cc = LU\F_cc
        Δ = Δ_aff + Δ_cc # final direction
        
        α_pri = compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        # new parameters
        x += α_pri * Δ[1:n_cols]
        λ += α_dual_final * Δ[n_cols+1: n_rows+n_cols]
        s_l[ilow] += α_dual_final * Δ[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u[iupp] += α_dual_final * Δ[n_rows+n_cols+n_low+1: end]
        
        X_low = Diagonal(x[ilow])
        for j=1:n_low
            S_low[j, ilow[j]] = s_l[ilow[j]]
        end
        X_upp = Diagonal(x[iupp])
        for j=1:n_upp
            S_upp[j, iupp[j]] = s_u[iupp[j]]
        end
        
        n_Δx = α_pri * norm(Δ[1:n_cols])
        μ = compute_μ(x[ilow], x[iupp], 
                      s_l[ilow], s_u[iupp], 
                      lvar[ilow], uvar[iupp],
                      n_low, n_upp)
        M11x = M11 * x
        xTM11x = x' * M11x
        q1Tx = q1' * x
        pri_obj = xTM11x/2 + q1Tx + c0  # no sense for a mLCP?
        dual_obj = -q2' * λ - xTM11x/2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0

        r1 = -M11x + M12 * λ + s_l - s_u - q1
        r2 = M21 * x + q2 - M22 * λ
        
        # update stopping criterion values:
        
        pdd = abs(pri_obj - dual_obj ) / (1 + abs(pri_obj)) 
        cond_r1 = norm(r1) / n_1_p1
        cond_r2 = norm(r2) / n_2_p1
        optimal = pdd < ϵ && cond_r2 < ϵ && cond_r1 < ϵ
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1
        
        if display == true
            @printf("%4d  % 9.2e   %7.1e    %7.1e    %7.1e    %7.1e   %7.1e\n", 
                    k, pri_obj, pdd, cond_r2, cond_r1,n_Δx , μ)
        end
        

    end
    
    if display == true
        criteria = [k >= max_iter,  optimal, small_Δx, small_μ]
        criteria_names = ["reached max_iter",  "optimal", 
            "n_Δx <= small_Δx", "μ <= ϵ_μ"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    elapsed_time = time() - start_time
    
    if k>= max_iter
        status = :max_iter
    else
        status = :acceptable
    end
    
"""    stats = GenericExecutionStats(status, QM, solution = x,
                                  objective = pri_obj , 
                                  dual_feas = cond_r1 * n_1_p1, 
                                  primal_feas = cond_r2 * n_2_p1,
                                  multipliers = λ,
                                  multipliers_L = s_l,
                                  multipliers_U = s_u,
                                  iter = k, elapsed_time=elapsed_time)"""
    stats = Dict()
    stats["objective"] = pri_obj
    stats["dual_feas"] = cond_r1 * n_1_p1
    stats["primal_feas"] = cond_r2 * n_2_p1
    stats["iter"] = k
    stats["elapsed_time"] = elapsed_time
    stats["solution"] = x
    return stats
end


mehrotraPCQuadBounds (generic function with 1 method)

In [39]:
# probleme1
Q = [6. 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1. 0 1
    0 1 1]
b = [0.; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

n_x, n_λ = 3, 2


x01 = [1.; 2.; 3.];


mLCP1 = mLCP(c0, c, -b, Q, A', A, [1 2; 3 19], lvar, uvar )

mLCP(0.0, [-8, -3, -3], [-0.0, -3.0], [6.0 2.0 1.0; 2.0 5.0 2.0; 1.0 2.0 4.0], [1.0 0.0; 0.0 1.0; 1.0 1.0], [1.0 0.0 1.0; 0.0 1.0 1.0], [1 2; 3 19], [0, 0, 0], [Inf, Inf, Inf])

In [40]:
stats_mpc1 =  mehrotraPCQuadBounds(mLCP1, 20)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0   7.00e+00   7.5e-01    4.0e+01    4.3e-01    0.0e+00   1.3e+00
   1   9.36e-01   1.4e+01    1.7e+00    3.7e-01    2.4e+00   4.9e-01
   2  -2.19e+00   5.6e+00    2.2e-16    3.6e-16    4.8e-01   7.1e-03
   3  -2.19e+00   5.6e+00    2.4e-09    5.5e-09    2.7e-03   7.1e-06
   4  -2.19e+00   5.6e+00    2.4e-12    5.5e-12    2.7e-06   7.1e-09

 stopping criterion = ["μ <= ϵ_μ"]


Dict{Any,Any} with 6 entries:
  "dual_feas"    => 5.55384e-11
  "primal_feas"  => 9.63163e-12
  "objective"    => -2.19427
  "elapsed_time" => 0.18
  "solution"     => [1.9521, 2.21662e-9, 1.29341]
  "iter"         => 4

In [33]:
Avals

UndefVarError: UndefVarError: Avals not defined

In [44]:
typeof(Q)

Array{Float64,2}